Cell 1 — Mount Drive

In [1]:
# 1) Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Rather than wrestling with token/branch-protection quirks, the cleanest path is to switch to the official GitHub Pages pipeline (no push to gh-pages; it uploads an artifact and deploys it).

What changes

Use actions/upload-pages-artifact + actions/deploy-pages.

Give the workflow pages: write and id-token: write permissions.

In your repo Settings → Pages, set Source = GitHub Actions (not Branch).

One-cell patch: write the new workflow, commit, push

Paste this in a new Colab cell in your repo folder and run it.

In [2]:
# Replace deploy workflow with GitHub Pages "deploy-pages" pipeline
PROJECT_DIR = "/content/drive/MyDrive/WorstellWitnessWaybackMachine"

from pathlib import Path
import textwrap, subprocess, os
os.chdir(PROJECT_DIR)

wf = textwrap.dedent("""
name: Deploy MkDocs site

on:
  push:
    branches: [ "main" ]
  workflow_dispatch:

permissions:
  contents: read
  pages: write
  id-token: write

concurrency:
  group: "pages"
  cancel-in-progress: true

jobs:
  build:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - uses: actions/setup-python@v5
        with:
          python-version: '3.x'
      - name: Install & build
        run: |
          python -m pip install --upgrade pip
          pip install mkdocs mkdocs-material
          mkdocs build --strict --site-dir site
      - name: Upload Pages artifact
        uses: actions/upload-pages-artifact@v3
        with:
          path: site

  deploy:
    environment:
      name: github-pages
      url: ${{ steps.deploy.outputs.page_url }}
    needs: build
    runs-on: ubuntu-latest
    steps:
      - name: Deploy to GitHub Pages
        id: deploy
        uses: actions/deploy-pages@v4
""").strip()

Path(".github/workflows").mkdir(parents=True, exist_ok=True)
Path(".github/workflows/deploy.yml").write_text(wf, encoding="utf-8")

def sh(cmd):
    print("$", cmd);
    return subprocess.call(cmd, shell=True)

sh('git add .github/workflows/deploy.yml')
sh('git commit -m "Switch to official GitHub Pages deploy workflow" || true')
sh('git push -u origin main')
print("✅ Workflow pushed. Now ensure Settings → Pages → Source = GitHub Actions.")

$ git add .github/workflows/deploy.yml
$ git commit -m "Switch to official GitHub Pages deploy workflow" || true
$ git push -u origin main
✅ Workflow pushed. Now ensure Settings → Pages → Source = GitHub Actions.


Then

In GitHub: Settings → Pages → Source = GitHub Actions.

Push any change (or re-run the workflow from Actions) → it should build and deploy without touching gh-pages.

Your site URL stays the same: https://BillWorstell.github.io/WorstellWitnessWaybackMachine/.